In [9]:
import arviz
import numpy as np
import os

In [32]:
def summarize_and_save(file, func_dict):
    """
    Calculates and saves summary for trace data.
    :param file: path where .nc file is located
    :param func_dict: dictionary defining which statistics should be contained in the summary
    :return: None
    """
    
    # define path where resulting summary is saved
    if file.endswith("trace.nc"):
        save_path = file[:-8] + "summary.csv"
        
        # read data and extract posterior
        idata = arviz.from_netcdf(file)
        posterior = idata.posterior.stack(sample=("chain", "draw"))

        # calculate summary
        summary = arviz.summary(
            idata,
            var_names=["r_t"],
            stat_funcs=func_dict,
            extend=False
        )
        summary = summary.set_index(np.array(posterior.date))

        # save as .csv
        summary.to_csv(save_path)
        
    else:
        print("Please pass trace data to return summary.")

In [44]:
folder = "data/"

func_dict = {
    "mean": np.mean,
    "std": np.std,
    "2.5%": lambda x: np.percentile(x, 2.5),
    "25%": lambda x: np.percentile(x, 25),
    "median": lambda x: np.percentile(x, 50),
    "75%": lambda x: np.percentile(x, 75),
    "97.5%": lambda x: np.percentile(x, 97.5),
}

In [45]:
for file in os.listdir(folder):
    summarize_and_save(folder + file, func_dict)